Intro 

Application for a work-study program at the Ecole Simplon


Scenario

Let's imagine that you're a system administrator working on a new project
assist a⸱data engineer, within a digital services SME.

Business objective:

This project concerns a customer who wants to be able to analyze the dynamics of their sales over time and space in order to improve their strategic decision-making.

Project Tree Stucture


Initial desired tree structure


In [ ]:
from IPython.display import Image, display

display(Image(filename="src/Arborescence-V2.png"))

In [ ]:
Path problem: to save time, all files are at the root except the src folder.

In [ ]:
from IPython.display import Image, display

display(Image(filename="src/Arborescence-V1.png"))

Part 1 : ARCHITECTURAL CONCEPTION

I - ARCHITECTURE OF THE 2-SERVICE ENVIRONMENT

On GitHub Codespaces, the Linux kernel used is managed by GitHub and is based on a customized Ubuntu image running on Azure virtual machines. It is not possible to modify this kernel directly, as updates are controlled by GitHub.

-> 1st service: ScriptRunner (Dockerfile)
Python image

Docker container
Custom script execution

-> 2nd service: DataWarehouse - BDD (Docker Compose - note: no Dockerlife because no customization)
SQLite3 image

Data storage and management in a SQLite3 database

In [ ]:

display(Image(filename="src/architecture-scheme.png"))

II - Communication inputs and outputs between services, indicating

In [ ]:
display(Image(filename="src/services-communication.png"))

PORT

SQLite is not a server like MySQL or PostgreSQL.
SQLite runs locally (codespace), in a .sqlite or .db file.
It does not open any network ports.
It can be accessed directly from a Python script, a terminal in the container, etc.
So :
No need to expose a port with SQLite → everything happens locally, in the container or via shared volumes.

NB:
Codespace sharing: If an application or service is accessible via a port (for example, a web server on port 8000), it's possible to share via this port.

Part 2 : ARCHITECTURAL CONCEPTION

I- Dockerlife

Service 1 (S1) : ScriptRunner (Dockerfile) 

> In terminal Launch Docker container :

docker build -t my-python-app .

> Once the image has been built: my-python-app 
Launch a container from this image ( my-python-app ) use the following command to launch the container :

docker run -it my-python-app /bin/bash

> Run the python script hello-world.py :

root@93c5907fe1c7:/app# python hello-world.py
Output: hello-world S1

I tried but failed to : Automate these 2 commands when opening Codespaces with a : 
Build the my-python-app image then launch an interactive container (/bin/bash) from this image.
> Create .devcontainer folder containing :n Dockerfile and devcontainer.json
Codespaces already uses a container as a development environment, built from your Dockerfile. So the container you want to launch with docker run is actually your Codespaces environment itself.

II- Docker Compose

Launch services

1- Start Docker Compose in the terminal

docker-compose up --build

2 -  To start the service that stores the data, use a Docker image with SQLite3
(you can find one on Docker Hub) and make sure that it works when you
you launch it locally (for example, you can access the
SQLite command line in the container).

Code : services:
  sqlite-db:
    image: nouchka/sqlite3:latest # Uses SQLite3 image
    stdin_open: true # Opens an interactive session
    tty: true # To keep the session interactive
    volumes:
      - sqlite-data:/data # Persistent volume for SQLite data

3- Launch Docker Compose container :

docker-compose up -d sqlite-db 
(The -d option launches the container in the background)

4- Access the Docker Compose container
Once the container is running, connect to its terminal with this command :
bash

docker exec -it project-sqlite-db-1 sh
Note:  docker ps command to find NAME db

5 - Launch SQLite from the command line
Once inside the container, type this command to launch SQLite ((sqlite>):
bash
sqlite3 /data/test.db


6 -  Quit SQLite and the container
To quit SQLite, type: .exit

To quit the container, simply type: bash exit


Note: Docker crashes => codespace relaunch
https://docs.docker.com/desktop/setup/install/linux/

Part 3 - PRELIMINARY DATA ANALYSIS

Part 3 - DATA ANALYZING


Problem encountered : 
- Not enough time to debug my script
- Periodicity or real-time import of customer data
- Problem: Extract customer csv files
- Connect to an API for city/region correspondence
- Field name (nomad convention with customer)

In [ ]:
from IPython.display import Image, display

display(Image(filename="src/ERD.png"))

Brief instruction : 
In the “hello-world” script (you can rename it if you like), integrate the logic to create the following
create the following elements:
a. The database
b. Tables 

*I executed this instruction as a comment in hello-world.py *

I - About de Data

- Source Data : 
The customer has provided you with an extract
extract of their 30-day sales dataset and data relating to their Products and Stores distributed in several French cities (Localization).

- Local import from user's terminal / Deposit on 

Too small sample to apply ROOOC methodology



II -Relationships between entities :

3 tables: Store, Product, Sales
Product → Sales :
Product.product_id → sales.product_id
A product may appear in several sales, but each sale concerns a single product 
=> Relation 1:N
1 : There is only one record in the first table.
N: This record can be associated with several records in the second table.

Store → Sales :
Store.store_id → sales.store_id
1:N
A store can have several sales, but each sale is made in a single store.

Product ↔ Store :
M:N via Sales table
A product can be sold in several stores, and a store can sell several products. This M:N relationship is managed via the Sales table.

M: “Several” records in the first table.
N: “Several” records in the second table.
Each record in the first table can be linked to several records in the second table, and each record in the second table can be linked to several records in the first table.

Relationship summary:
1:N: A product can appear in several sales, but each sale concerns a single product. 
1:N: A store can have several sales, but each sale belongs to a single store.
M:N: A product can be sold in several stores, and a store can sell several products, which is managed via the Sales link table.

III- Data import 

Method 1 => via ScriptRunner 
(+) fine granularity (processing, cleansing) , (-) very slow and inefficient on large volumes
Method 2 => DataWarehouse direct (csv easy to purge and format in our case beforehand) :
(+) Much faster: mass ingestion
(-) Possibility of directly connecting BI tools

According to Methodology 1 instruction (Q: in our case, considering the business challenges (Sales) a priori M2).

Business and workflow issues: periodic or real-time import of customer csvs

IV - Problems encountered : 
Periodicity or real-time import of customer data
Problem: Extract customer csv files
Connect to an API for city/region correspondence
Field name (nomad convention with customer)


a - Rebuiding images 

After creating sales-analyze.py and not found on app
I had to rebuild the app image: 

docker build -t my-python-app .
docker run my-python-app
docker run -it my-python-app /bin/bash (bash ls - the file is in the app)

I checked on the database Docker whether the bdd had been created correctly 

docker-compose up --build
docker-compose up -d sqlite-db 
(The -d option launches the container in the background)
docker-compose restart script-runner

docker exec -it project-sqlite-db-1 sh

b - Data cleaning and formatting 

Problems with inconsistent data or replacement by an average value or Nan or deletion of record line 
Question: as with R, avoid making loops so you don't have to do it for each df

c - Business and workflow issues
Periodic or real-time import of customer csvs
System load optimization: 
To avoid excessive power or resource consumption in Docker containers
If file is local, use python watchdog library (real-time monitoring of modifications)
Use a watchdog in a separate process: in the background or a separate service in Docker (e.g. a cron or a script) to perform the update, instead of continuously polling with a Python script.


ANALYSE RESULT VISUALIZATION

In [ ]:
!pip install pandas matplotlib seaborn

# Import necessary libraries for vizualisation
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
# Connect to SQLite database
db_name = "sales-analyze.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

Load Result table

In [ ]:
df = pd.read_sql_query("SELECT * FROM Result", conn)
conn.close()

# Data overview
df.head()

 Visualization (a) : Total Revenue

In [ ]:
total_revenue = df[df["type"] == "total_revenue"]["total_revenue"].sum()

import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5))
plt.bar(["Total Revenue"], [total_revenue], color="green")
plt.title("Total Revenue")
plt.ylabel("Amount (€)")
plt.show()

 Visualization (b) : Sales by Product

In [ ]:
df_products = df[df["type"] == "sales_by_product"]

plt.figure(figsize=(10, 6))
plt.bar(df_products["product_name"], df_products["total_revenue"], color="skyblue")
plt.title("Total Revenue by Product")
plt.xlabel("Product")
plt.ylabel("Revenue (€)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

 Visualization (c) : Sales by Region

In [ ]:
df_regions = df[df["type"] == "sales_by_region"]

import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(data=df_regions, x="region", y="total_revenue", palette="mako")
plt.title("Total Revenue by Region")
plt.xlabel("Region")
plt.ylabel("Revenue (€)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

APPENDIX

Areas for improvement : 


- Methods for centralizing sales data
(Google Drive, API, Integrator ( Zapier),
- Upload customer files via url 
(Make a utility that checks data loading: only dirty data because (secure Maj change product and stores )
- Others services : Visualization, Loading critical files as sales
- API data file (cleansing and transformation) - workflow optimization and security 


Git Commands:

Git COMMIT (qd le faire) pas fait de versioning
(Bash - Terminal)
git add .
git commit -m 
git push origin main

In [ ]:
!jupyter nbconvert --to html analyse_ventes.ipynb


Right-click on the .html file → “Download” to open it in your browser.